In [7]:
import pandas as pd
import numpy as np
import os

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
import warnings
warnings.filterwarnings('ignore')

## Read Data

In [166]:
old_df = pd.read_csv('nba_offense_old.csv')
old_df['Player'] = old_df['Player'].apply(lambda x: ' '.join(x.split(' ')[1:]))
old_df['Season'] = old_df['Season'].apply(lambda x: f'{x}-{x+1}')
old_df['Overall_Poss'] = old_df['Overall_Poss'] - old_df['Miscellaneous_Poss']
print(old_df.shape)
old_df.columns

(3675, 74)


Index(['Player', 'Overall_Poss', 'Overall_Points', 'Overall_PPP',
       'Overall_Rank', 'Overall_Rating', 'Team', 'Spot Up_Poss',
       'Spot Up_Points', 'Spot Up_PPP', 'Spot Up_Rank', 'Spot Up_Rating',
       'Transition_Poss', 'Transition_Points', 'Transition_PPP',
       'Transition_Rank', 'Transition_Rating', 'P&R Ball Handler_Poss',
       'P&R Ball Handler_Points', 'P&R Ball Handler_PPP',
       'P&R Ball Handler_Rank', 'P&R Ball Handler_Rating',
       'Offensive Rebounds (put backs)_Poss',
       'Offensive Rebounds (put backs)_Points',
       'Offensive Rebounds (put backs)_PPP',
       'Offensive Rebounds (put backs)_Rank',
       'Offensive Rebounds (put backs)_Rating', 'Cut_Poss', 'Cut_Points',
       'Cut_PPP', 'Cut_Rank', 'Cut_Rating', 'Hand Off_Poss', 'Hand Off_Points',
       'Hand Off_PPP', 'Hand Off_Rank', 'Hand Off_Rating', 'Isolation_Poss',
       'Isolation_Points', 'Isolation_PPP', 'Isolation_Rank',
       'Isolation_Rating', 'Off Screen_Poss', 'Off Screen_Point

In [51]:
old_df[old_df['Season'] == 2021].shape

(702, 74)

## Build Data

### Elements

In [82]:
playtypes = ['Isolation', 'P&R Ball Handler', 'Post-Up', 
             'P&R Roll Man', 'Spot Up', 'Off Screen', 
             'Hand Off', 'Cut', 'Offensive Rebounds', 
             'Transition', 
#              'Miscellaneous'
            ]

In [46]:
situations = ['Pick and Rolls Including Passes',
              'Isolations Including Passes',
              'Post-Ups Including Passes']

In [71]:
shottypes = ['Catch and Shoot', 'At Rim',
             'Long (3 point jump shots)',
             'All Jump Shots off the Dribble']

### Functions

In [62]:
def process_all_fg_sheet(df, season):
    df['Season'] = season
    df = df[['Player', 'Team', 'Season', 'GP', 'FG Made', 'FG Att']]\
         .rename(columns={'FG Made': 'FGM',
                          'FG Att': 'FGA'})
    df['GP'] = df['GP'].apply(int)
    for col in ['FGM', 'FGA']:
        df[col] = df[col].apply(np.float) * df['GP']
    return df.sort_values(['Team', 'FGA'], ascending=False)

In [63]:
def process_shottype_sheet(shottype, df, season):
    df['Season'] = season
    df = df[['Player', 'Team', 'Season', 'GP', 'FG Made', 'FG Att']]\
         .rename(columns={'FG Made': 'FGM',
                          'FG Att': 'FGA'})
    df['GP'] = df['GP'].apply(int)
    for col in ['FGM', 'FGA']:
        df[col] = df[col].apply(np.float) * df['GP']
    df['FG%'] = df['FGM'] / df['FGA'] * 1.0
    return df.rename(columns={col: f'{shottype}_{col}'
                              for col in df.columns
                              if 'FG' in col})

In [79]:
def process_playtype_sheet(playtype, df, season):
    df['Season'] = season
    df = df[['Player', 'Team', 'Season', 'GP', 'Poss', 'PPP']]
    df['GP'] = df['GP'].apply(int)
    df['PPP'] = df['PPP'].apply(np.float)
    df['Poss'] = df['Poss'].apply(np.float) * df['GP']
    return df.rename(columns={col: f'{playtype}_{col}'
                              for col in ['Poss', 'PPP']})

In [97]:
def process_situation_sheet(situation, df, season):
    df['Season'] = season
    df = df[['Player', 'Team', 'Season', 'GP', 'Poss']]
    df['GP'] = df['GP'].apply(int)
    df['Poss'] = df['Poss'].apply(np.float) * df['GP']
    return df.rename(columns={'Poss': f'{situation}_Poss'})

In [102]:
def process_season(league, season,
                   shottypes=shottypes, 
                   playtypes=playtypes,
                   situations=situations):
    season_dir = f'{league}_{season}'

    # Initialize
    for sheet_fn in os.listdir(season_dir):
        if 'All field goal attempts' in sheet_fn:
            df = pd.read_csv(f'{season_dir}/{sheet_fn}', 
                             header=1)
            break
    df = process_all_fg_sheet(df, season)
    print('Shape after Initialization:', df.shape)

    # Process shottype
    for shottype in shottypes:
        for sheet_fn in os.listdir(season_dir):
            if shottype in sheet_fn:
                st_df = pd.read_csv(f'{season_dir}/{sheet_fn}', 
                                    header=1)
                break
        st_df = process_shottype_sheet(shottype, st_df, season)
        df = df.merge(st_df, 
                      on=['Player', 'Team', 'Season', 'GP'],
                      how='left')\
               .fillna(0)
        df[f'{shottype}_FGA%'] = df[f'{shottype}_FGA'] / df['FGA'] * 1.0

    # Process playtype
    for playtype in playtypes:
        for sheet_fn in os.listdir(season_dir):
            if playtype in sheet_fn and 'Including Passes' not in sheet_fn:
                pt_df = pd.read_csv(f'{season_dir}/{sheet_fn}', 
                                    header=1)
                break
        pt_df = process_playtype_sheet(playtype, pt_df, season)
        df = df.merge(pt_df,
                      on=['Player', 'Team', 'Season', 'GP'],
                      how='left')\
               .fillna(0)
    df['Overall_Poss'] = df[[col for col in df.columns
                             if 'Poss' in col]].sum(axis=1)
    for playtype in playtypes:
        df[f'{playtype}_Poss%'] = df[f'{playtype}_Poss'] / df['Overall_Poss'] * 1.0

    # Process situations
    for situation in situations:
        for sheet_fn in os.listdir(season_dir):
            if situation in sheet_fn:
                sit_df = pd.read_csv(f'{season_dir}/{sheet_fn}', 
                                     header=1)
                break
        sit_df = process_situation_sheet(situation, sit_df, season)
        df = df.merge(sit_df,
                      on=['Player', 'Team', 'Season', 'GP'],
                      how='left')\
               .fillna(0)
    print('Shape after Processing:', df.shape)
    
    return df

### Process

In [187]:
league = 'NBA'
dfs = []
for start_year in range(2017, 2022):
    end_year = start_year + 1
    season = f'{start_year}-{end_year}'
    print(season)
    dfs.append(process_season(league, season))
    print()
    
df = pd.concat(dfs).reset_index().drop('index', axis=1)
print(df.shape)

2017-2018
Shape after Initialization: (543, 6)
Shape after Processing: (543, 56)

2018-2019
Shape after Initialization: (573, 6)
Shape after Processing: (573, 56)

2019-2020
Shape after Initialization: (549, 6)
Shape after Processing: (549, 56)

2020-2021
Shape after Initialization: (591, 6)
Shape after Processing: (591, 56)

2021-2022
Shape after Initialization: (611, 6)
Shape after Processing: (611, 56)

(2867, 56)


In [188]:
df.to_csv(f'{league}_offense.csv', index=False)